In [14]:
using HDF5, JLD, DataFrames, Distributions, MLBase, StatsBase, PlotlyJS, Mocha
include("MyFunctions.jl");

In [3]:
BLAS.set_num_threads(16)

In [4]:
fullDataSTD = load("dataV2.jld")["dataFilteredSTD"];
bigY = load("dataV2.jld")["bigY"];
Ysum = Array{Float64}(nrow(bigY),1);
Ysum = convert(Array{Float64}, bigY[:P_CW_P_ACW]);

In [48]:
#data_layer  = HDF5DataLayer(name="train-data", source="mnist/train.txt",
#    batch_size=64, shuffle=true)
#MemoryDataLayer(tops=[:data, :label], batch_size=1000, data=Array[fullDataSTD[bernou,:], Ysum[bernou]])
#data_tt_layer = AsyncHDF5DataLayer(name="data-test", source="gDataHDF5/test.txt", batch_size=100)

# prepare Mocha network with similar architecture, but with additional layers
# for data and softmax classification
# names of layers should be the same as in DBN
bernou = rand(Bernoulli(0.9), size(Ysum)) .== 1;
batch_sz = 1000
data = MemoryDataLayer(tops=[:data, :label], batch_size=batch_sz, data=Array[fullDataSTD[bernou,:]', Ysum[bernou,:]'])
hid1 = InnerProductLayer(name="hid1", output_dim=500, tops=[:hid1], bottoms=[:data], neuron=Neurons.Sigmoid())
hid2 = InnerProductLayer(name="hid2", output_dim=500, tops=[:hid2], bottoms=[:hid1], neuron=Neurons.Sigmoid())
hid4 = InnerProductLayer(name="hid4", output_dim=1, tops=[:hid4], bottoms=[:hid2], neuron=Neurons.Sigmoid())
loss = SquareLossLayer(name="loss", bottoms=[:hid4, :label])

backend = DefaultBackend()
init(backend)

net = Net("Spirality-Train", backend, [data, hid1, hid2, hid4, loss])

17-Nov 18:30:11:INFO:root:Constructing net Spirality-Train on Mocha.CPUBackend...
17-Nov 18:30:11:INFO:root:Topological sorting 5 layers...
17-Nov 18:30:11:INFO:root:Setup layers...
17-Nov 18:30:11:INFO:root:Network constructed!


************************************************************
          NAME: Spirality-Train
       BACKEND: Mocha.CPUBackend
  ARCHITECTURE: 5 layers
............................................................
 *** Mocha.MemoryDataLayer(memory-data)
    Outputs ---------------------------
          data: Blob(181 x 1000)
         label: Blob(1 x 1000)
............................................................
 *** Mocha.InnerProductLayer(hid1)
    Inputs ----------------------------
          data: Blob(181 x 1000)
    Outputs ---------------------------
          hid1: Blob(500 x 1000)
............................................................
 *** Mocha.InnerProductLayer(hid2)
    Inputs ----------------------------
          hid1: Blob(500 x 1000)
    Outputs ---------------------------
          hid2: Blob(500 x 1000)
............................................................
 *** Mocha.InnerProductLayer(hid4)
    Inputs ----------------------------
          hid2: Blob(500

In [49]:
exp_dir = "SPNNsnapshots100_50_25_DV2"
method = Adam()
params = make_solver_parameters(method, max_iter=10000, regu_coef=0.0005,
    #mom_policy=MomPolicy.Fixed(0.99),
    lr_policy=LRPolicy.Inv(0.01, 0.0001, 0.75),
    load_from=exp_dir)
solver = Solver(method, params)

Mocha.Solver{Mocha.Adam}(Mocha.Adam(),Dict{Symbol,Any}(Pair{Symbol,Any}(:regu_coef,0.0005),Pair{Symbol,Any}(:load_from,"SPNNsnapshots100_50_25_DV2"),Pair{Symbol,Any}(:beta1,0.9),Pair{Symbol,Any}(:lr_policy,Mocha.LRPolicy.Inv(0.01,0.0001,0.75)),Pair{Symbol,Any}(:beta2,0.999),Pair{Symbol,Any}(:max_iter,10000),Pair{Symbol,Any}(:epsilon,1.0e-8)),Mocha.CoffeeLounge("",1,:merge,Dict{AbstractString,Dict{Int64,AbstractFloat}}(),Mocha.CoffeeBreak[],true,8751724873810518572,8247343408463635824))

In [50]:
setup_coffee_lounge(solver, save_into="$exp_dir/statistics.jld", every_n_iter=1000)

# report training progress every 100 iterations
add_coffee_break(solver, TrainingSummary(), every_n_iter=100)

# save snapshots every 5000 iterations
add_coffee_break(solver, Snapshot(exp_dir), every_n_iter=5000)


2-element Array{Mocha.CoffeeBreak,1}:
 Mocha.CoffeeBreak(Mocha.TrainingSummary(Any[:iter,:obj_val]),100,0)   
 Mocha.CoffeeBreak(Mocha.Snapshot("SPNNsnapshots100_50_25_DV2"),5000,0)

In [51]:
# show performance on test data every 1000 iterations
data_layer_test = MemoryDataLayer(tops=[:data, :label], batch_size=batch_sz, data=Array[fullDataSTD[!bernou,:]', Ysum[!bernou,:]'])
test_net = Net("Spirality-Test", backend,  [data_layer_test, hid1, hid2, hid4, loss])
add_coffee_break(solver, ValidationPerformance(test_net), every_n_iter=1000);

17-Nov 18:30:12:INFO:root:Constructing net Spirality-Test on Mocha.CPUBackend...
17-Nov 18:30:12:INFO:root:Topological sorting 5 layers...
17-Nov 18:30:12:INFO:root:Setup layers...
17-Nov 18:30:12:DEBUG:root:InnerProductLayer(hid1): sharing weights and bias
17-Nov 18:30:12:DEBUG:root:InnerProductLayer(hid2): sharing weights and bias
17-Nov 18:30:12:DEBUG:root:InnerProductLayer(hid4): sharing weights and bias
17-Nov 18:30:12:INFO:root:Network constructed!


In [52]:
@time solve(solver, net)

17-Nov 18:30:12:DEBUG:root:#DEBUG Checking network topology for back-propagation
17-Nov 18:30:12:DEBUG:root:Init network Spirality-Train
17-Nov 18:30:12:DEBUG:root:Init parameter weight for layer hid1
17-Nov 18:30:12:DEBUG:root:Init parameter bias for layer hid1
17-Nov 18:30:12:DEBUG:root:Init parameter weight for layer hid2
17-Nov 18:30:12:DEBUG:root:Init parameter bias for layer hid2
17-Nov 18:30:12:DEBUG:root:Init parameter weight for layer hid4
17-Nov 18:30:12:DEBUG:root:Init parameter bias for layer hid4
17-Nov 18:30:12:DEBUG:root:#DEBUG Initializing coffee breaks
17-Nov 18:30:12:INFO:root:Snapshot directory SPNNsnapshots100_50_25_DV2 already exists
17-Nov 18:30:12:DEBUG:root:Init network Spirality-Test
17-Nov 18:30:12:INFO:root: TRAIN iter=000000 obj_val=0.03462924
17-Nov 18:30:12:INFO:root:Saving snapshot to snapshot-000000.jld...
17-Nov 18:30:12:DEBUG:root:Saving parameters for layer hid1
17-Nov 18:30:12:DEBUG:root:Saving parameters for layer hid2
17-Nov 18:30:12:DEBUG:root:Sav

3-element Array{Array{Void,1},1}:
 Void[nothing,nothing]
 Void[nothing,nothing]
 Void[nothing,nothing]

In [53]:
results = MemoryOutputLayer(bottoms=[:hid4])
test_net_results = Net("Spirality-Test", backend, [data_layer_test, hid1, hid2, hid4, results])

17-Nov 18:41:58:INFO:root:Constructing net Spirality-Test on Mocha.CPUBackend...
17-Nov 18:41:58:INFO:root:Topological sorting 5 layers...
17-Nov 18:41:58:INFO:root:Setup layers...
17-Nov 18:41:58:DEBUG:root:InnerProductLayer(hid1): sharing weights and bias
17-Nov 18:41:58:DEBUG:root:InnerProductLayer(hid2): sharing weights and bias
17-Nov 18:41:58:DEBUG:root:InnerProductLayer(hid4): sharing weights and bias
17-Nov 18:41:58:INFO:root:Network constructed!


************************************************************
          NAME: Spirality-Test
       BACKEND: Mocha.CPUBackend
  ARCHITECTURE: 5 layers
............................................................
 *** Mocha.MemoryDataLayer(memory-data)
    Outputs ---------------------------
          data: Blob(181 x 1000)
         label: Blob(1 x 1000)
............................................................
 *** Mocha.InnerProductLayer(hid1)
    Inputs ----------------------------
          data: Blob(181 x 1000)
    Outputs ---------------------------
          hid1: Blob(500 x 1000)
............................................................
 *** Mocha.InnerProductLayer(hid2)
    Inputs ----------------------------
          hid1: Blob(500 x 1000)
    Outputs ---------------------------
          hid2: Blob(500 x 1000)
............................................................
 *** Mocha.InnerProductLayer(hid4)
    Inputs ----------------------------
          hid2: Blob(500 

In [54]:
forward_epoch(test_net_results)

In [55]:
predictions = Float64[]
for i in 1:cld(length(Ysum[!bernou,:]),batch_sz)
    if i == cld(length(Ysum[!bernou,:]),batch_sz)
        last = length(Ysum[!bernou,:]) - fld(length(Ysum[!bernou,:]),batch_sz)*batch_sz
        predictions = vcat(predictions, (test_net_results.states[end].outputs[1][i]')[1:last])
    else
        predictions = vcat(predictions, test_net_results.states[end].outputs[1][i]')
    end
end

In [56]:
println("RMSE: $(root_mean_squared_error(Ysum[!bernou,:],predictions))")
println("Pearson Correlation: $(cor(Ysum[!bernou,:],predictions)[1])")

RMSE: 0.15760253226941115
Pearson Correlation: 0.7291777474097891


In [38]:
destroy(net)
destroy(test_net)
destroy(test_net_results)
shutdown(backend)

17-Nov 02:27:40:DEBUG:root:Destroying network Spirality-Train
17-Nov 02:27:40:DEBUG:root:Destroying network Spirality-Test
17-Nov 02:27:40:DEBUG:root:Destroying network Spirality-Test


Dict{AbstractString,Array{Mocha.AbstractParameter,1}} with 0 entries